## HPP - project

## Setup

In [36]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
from sklearn.preprocessing import OneHotEncoder
import xgboost

# read in all our data
HPP_data = pd.read_csv("train.csv")
# set seed for reproducibility
np.random.seed(0)

## Filling missing values in the train data

In [37]:
#Get the number of missing data points per column
missing_values_count = HPP_data.isnull().sum()
#Get columns with at least one missing data point
columns_with_missing_data = missing_values_count[missing_values_count > 0]

Heatmap - find out which to features to drop

In [38]:
# Numeric
HPP_data['LotFrontage'] = HPP_data['LotFrontage'].fillna(HPP_data['LotFrontage'].mean())
HPP_data['MasVnrArea'] = HPP_data['MasVnrArea'].fillna(HPP_data['MasVnrArea'].mean())
HPP_data['GarageYrBlt'] = HPP_data['GarageYrBlt'].fillna(HPP_data['GarageYrBlt'].mean())

# Categorical
HPP_data['Alley'] = HPP_data['Alley'].fillna(HPP_data['Alley'].mode()[0])
HPP_data['MasVnrType'] = HPP_data['MasVnrType'].fillna(HPP_data['MasVnrType'].mode()[0])
HPP_data['BsmtQual'] = HPP_data['BsmtQual'].fillna(HPP_data['BsmtQual'].mode()[0])
HPP_data['BsmtCond'] = HPP_data['BsmtCond'].fillna(HPP_data['BsmtCond'].mode()[0])
HPP_data['BsmtExposure'] = HPP_data['BsmtExposure'].fillna(HPP_data['BsmtExposure'].mode()[0])
HPP_data['BsmtFinType1'] = HPP_data['BsmtFinType1'].fillna(HPP_data['BsmtFinType1'].mode()[0])
HPP_data['BsmtFinType2'] = HPP_data['BsmtFinType2'].fillna(HPP_data['BsmtFinType2'].mode()[0])
HPP_data['Electrical'] = HPP_data['Electrical'].fillna(HPP_data['Electrical'].mode()[0])
HPP_data['FireplaceQu'] = HPP_data['FireplaceQu'].fillna(HPP_data['FireplaceQu'].mode()[0])
HPP_data['GarageType'] = HPP_data['GarageType'].fillna(HPP_data['GarageType'].mode()[0])
HPP_data['GarageFinish'] = HPP_data['GarageFinish'].fillna(HPP_data['GarageFinish'].mode()[0])
HPP_data['GarageQual'] = HPP_data['GarageQual'].fillna(HPP_data['GarageQual'].mode()[0])
HPP_data['GarageCond'] = HPP_data['GarageCond'].fillna(HPP_data['GarageCond'].mode()[0])
HPP_data['PoolQC'] = HPP_data['PoolQC'].fillna(HPP_data['PoolQC'].mode()[0])
HPP_data['Fence'] = HPP_data['Fence'].fillna(HPP_data['Fence'].mode()[0])
HPP_data['MiscFeature'] = HPP_data['MiscFeature'].fillna(HPP_data['MiscFeature'].mode()[0])


# Check that we have no NA values.
missing_values_count = HPP_data.isnull().sum()
columns_with_missing_data = missing_values_count[missing_values_count > 0]
print(columns_with_missing_data)

Series([], dtype: int64)


Handle Categorical Features

## Test data

## Concatination

In [39]:
# HPP_data_test = pd.read_csv('formulatedtest.csv')
# HPP_data = pd.concat([HPP_data,HPP_data_test],axis=0)

## Encoding

In [40]:
# Identify categorical columns
categorical_columns = HPP_data.select_dtypes(include=['object']).columns
# Preprocess the data to ensure categorical columns contain only strings
HPP_data[categorical_columns] = HPP_data[categorical_columns].astype(str)
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Fit and transform the categorical data using one-hot encoding
X_encoded = encoder.fit_transform(HPP_data[categorical_columns])
# Get the feature names
feature_names = encoder.get_feature_names_out(input_features=categorical_columns)
# Create a DataFrame with the one-hot encoded features
X_encoded_HPP_data = pd.DataFrame(X_encoded, columns=feature_names)
# Combine the one-hot encoded features with the original numerical features
HPP_data = pd.concat([X_encoded_HPP_data, HPP_data.drop(categorical_columns, axis=1)], axis=1)
# X_final.shape

# TODO: Might be duplicates
# X_final = X_final.loc[:,~X_final.columns.duplicated()]
# X_final.shape

## Split

In [41]:
HPP_data_Train = HPP_data.iloc[:1422,:]
HPP_data_Test = HPP_data.iloc[1422:,:]
HPP_data_Test.drop(['SalePrice'],axis=1,inplace=True)


# HPP_data_Test.drop(['SalePrice'],axis=1,inplace=True)
X_train= HPP_data_Train.drop(['SalePrice'],axis=1)
y_train= HPP_data_Train['SalePrice']

/var/folders/vk/b9g86spn6n9d9mny96n4mww80000gn/T/ipykernel_4867/2500699194.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HPP_data_Test.drop(['SalePrice'],axis=1,inplace=True)


In [42]:
classifier = xgboost.XGBRegressor()
classifier.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [43]:
# import pickle
# filename = 'finalized_model.pkl'
# pickle.dump(classifier, open(filename, 'wb'))

In [44]:
y_pred = classifier.predict(HPP_data_Test)